# Chapter 9

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [2]:
from statsmodels.stats.diagnostic import linear_reset

# Exercise 1
ceosal1 = pd.read_stata("stata/CEOSAL1.DTA")
ceosal1["rosneg"] = (ceosal1.ros < 0).astype('int32')

y = ceosal1.lsalary
X = sm.add_constant(ceosal1[["lsales", "roe", "rosneg"]])
model = sm.OLS(y, X).fit()
print(linear_reset(model))

<Wald test (chi2): statistic=[[2.66707016]], p-value=0.2635439634709704, df_denom=2>


In [3]:
linear_reset(model, cov_type = "HC3")

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[2.99859337]], p-value=0.22328714600347846, df_denom=2>

In [4]:
ceosal1["lsales_sq"] = np.power(model.fittedvalues, 2)
ceosal1["roe_sq"] = np.power(model.fittedvalues, 3)

In [5]:
quad_model = sm.OLS(y, sm.add_constant(ceosal1[["lsales", "roe", "rosneg", "lsales_sq", "roe_sq"]])).fit()

In [6]:
quad_model.f_test("lsales_sq = roe_sq = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.33353508]]), p=0.2658425972281826, df_denom=203, df_num=2>

In [7]:
linear_reset(model, use_f = True)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.33353508]]), p=0.26584259722822173, df_denom=203, df_num=2>

In [8]:
quad_model.get_robustcov_results().f_test("lsales_sq = roe_sq = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[2.17447904]]), p=0.11630792921312504, df_denom=203, df_num=2>

In [9]:
quad_model.get_robustcov_results(cov_type = "HC3").f_test("lsales_sq = roe_sq = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.49929669]]), p=0.22574907479133416, df_denom=203, df_num=2>

In [10]:
linear_reset(model, cov_type = "HC3", use_f = True)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.49929669]]), p=0.22574907479139664, df_denom=203, df_num=2>